Binomial

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

def binomial_model(S0, K, T, r, sigma, N, option_type='call'):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))  # Up factor
    d = 1 / u                        # Down factor
    p = (np.exp(r * dt) - d) / (u - d)  # Risk-neutral probability

    ST = np.zeros(N + 1)
    for i in range(N + 1):
        ST[i] = S0 * (u ** (N - i)) * (d ** i)

    if option_type == 'call':
        option = np.maximum(ST - K, 0)
    elif option_type == 'put':
        option = np.maximum(K - ST, 0)
    else:
        raise ValueError("option_type must be 'call' or 'put'")

    for j in range(N - 1, -1, -1):
        for i in range(j + 1):
            option[i] = np.exp(-r * dt) * (p * option[i] + (1 - p) * option[i + 1])

    return option[0]

# Download data
data = yf.download('AAPL', start='2023-01-01', end='2024-01-01')

K = 180.0      # Strike price
T = 30 / 365   # Time to maturity (30 days)
r = 0.05       # Risk-free rate (5%)
sigma = 0.25   # Volatility (25%)
N = 100        # Number of time steps

entry_dates = []
exit_dates = []
profits = []

for i in range(len(data) - 1):
    S0 = data['Close'].iloc[i]
    entry_date = data.index[i]
    binomial_price = binomial_model(S0, K, T, r, sigma, N, option_type='call')

    if binomial_price < 2.0:
        entry_dates.append(entry_date)
        exit_index = min(i + 10, len(data) - 1)
        exit_date = data.index[exit_index]
        S1 = data['Close'].iloc[exit_index]
        exit_price = binomial_model(S1, K, T, r, sigma, N, option_type='call')
        profits.append(exit_price - binomial_price)
        exit_dates.append(exit_date)

trade_details_binomial = pd.DataFrame({
    "Entry Date": entry_dates,
    "Exit Date": exit_dates,
    "Profit": profits
})

trade_details_binomial.to_csv('binomial_trade_details.csv', index=False)


/tmp/ipython-input-11-1792675206.py:29: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('AAPL', start='2023-01-01', end='2024-01-01')
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-11-1792675206.py:13: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  ST[i] = S0 * (u ** (N - i)) * (d ** i)


Black Scholes

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import norm

def black_scholes(S, K, T, r, sigma, option_type='call'):
    if T <= 0:
        # Option expired; payoff is intrinsic value
        if option_type == 'call':
            return max(S - K, 0)
        elif option_type == 'put':
            return max(K - S, 0)
        else:
            raise ValueError("option_type must be 'call' or 'put'")

    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("option_type must be 'call' or 'put'")

    return price

# Download historical data, specifying auto_adjust=False to suppress warning
data = yf.download('AAPL', start='2023-01-01', end='2024-01-01', auto_adjust=False)

# Drop missing values if any
data = data.dropna()

K = 180.0       # Strike price
T = 30 / 365    # Time to maturity (30 days)
r = 0.05        # Risk-free rate (5%)
sigma = 0.25    # Volatility (25%)

entry_dates = []
exit_dates = []
profits = []

for i in range(len(data) - 1):
    S0 = float(data['Close'].iloc[i])   # ensure scalar float
    entry_date = data.index[i]
    bs_price = black_scholes(S0, K, T, r, sigma, option_type='call')

    if bs_price < 2.0:
        entry_dates.append(entry_date)
        exit_index = min(i + 10, len(data) - 1)
        exit_date = data.index[exit_index]
        S1 = float(data['Close'].iloc[exit_index])  # ensure scalar float
        exit_price = black_scholes(S1, K, T, r, sigma, option_type='call')
        profits.append(exit_price - bs_price)
        exit_dates.append(exit_date)

trade_details_black_scholes = pd.DataFrame({
    'Entry Date': entry_dates,
    'Exit Date': exit_dates,
    'Profit': profits
})

trade_details_black_scholes.to_csv('black_scholes_trade_details.csv', index=False)

[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-9-1750595104.py:44: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  S0 = float(data['Close'].iloc[i])   # ensure scalar float
/tmp/ipython-input-9-1750595104.py:52: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  S1 = float(data['Close'].iloc[exit_index])  # ensure scalar float


Monte Carlo

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd

def monte_carlo_simulation(S0, K, T, r, sigma, M, I):
    dt = T / M
    Z = np.random.standard_normal((M, I))
    S = np.zeros((M + 1, I))
    S[0] = S0
    for t in range(1, M + 1):
        S[t] = S[t - 1] * np.exp((r - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * Z[t - 1])
    payoff = np.maximum(S[-1] - K, 0)
    option_price = np.exp(-r * T) * np.mean(payoff)
    return option_price

# Download historical data for AAPL
data = yf.download('AAPL', start='2023-01-01', end='2024-01-01')

K = 180.0       # Strike price
T = 30 / 365    # Time to maturity (30 days)
r = 0.05        # Risk-free rate (5%)
sigma = 0.25    # Volatility (25%)
M = 100         # Number of time steps
I = 100000      # Number of simulations

entry_dates = []
exit_dates = []
profits = []

for i in range(len(data) - 1):
    S0 = data['Close'].iloc[i]
    entry_date = data.index[i]
    monte_carlo_price = monte_carlo_simulation(S0, K, T, r, sigma, M, I)

    if monte_carlo_price < 2.0:
        entry_dates.append(entry_date)
        exit_idx = min(i + 10, len(data) - 1)
        exit_date = data.index[exit_idx]
        S1 = data['Close'].iloc[exit_idx]
        exit_price = monte_carlo_simulation(S1, K, T, r, sigma, M, I)
        profits.append(exit_price - monte_carlo_price)
        exit_dates.append(exit_date)

# Create DataFrame of trade details
trade_details_monte_carlo = pd.DataFrame({
    'Entry Date': entry_dates,
    'Exit Date': exit_dates,
    'Profit': profits
})

trade_details_monte_carlo.to_csv('monte_carlo_trade_details.csv', index=False)


/tmp/ipython-input-10-977538005.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('AAPL', start='2023-01-01', end='2024-01-01')
[*********************100%***********************]  1 of 1 completed
